In [2]:
%pylab inline
import pandas as pd
from numpy.linalg import matrix_rank
from numpy.linalg import pinv as moore_penrose_inverse
from numpy.linalg import svd

Populating the interactive namespace from numpy and matplotlib


In [217]:
def GSVD(A, B):
    """ Implements GSVD from Paige and Saunders (1981)
    
    Parameters
    ----------
    A, B: array_like
          Input Matrix
          
    Returns
    -------
    
    U: array_like
       
    V: array_like
    

       
    """
    assert A.shape[1] == B.shape[1], 'A, B need same number of columns'
    m, n = A.shape
    p, n = B.shape
    
    # C.T = (A.T, B.T)
    C = np.vstack([A, B])
    
    # Step 1
    # P.T C Q = (R 0; 0 0)
    P, R, QT = svd(C)
    
    nonzero_sigma = R[R>1e-12]
    # r = matrix_rank(C)
    r = len(nonzero_sigma)
    l = matrix_rank(B)

    P11 = P[:m, :r]
    P21 = P[m:m+p, :r]

    U, Sigma_A, W1 = svd(P11, full_matrices=True)    
    V, Sigma_B, W2 = svd(P21, full_matrices=True)    

    kr = min([p, r])
    k = r - l
    S_B = np.zeros((p, r))
    
    indices1 = np.arange(p-1,  p-kr-1, -1)
    indices2 = np.arange(r-1, r-kr-1, -1)
    values = Sigma_B[np.arange(0, kr)]
    S_B[indices1, indices2] = list(values)
    
    V = np.dot(np.dot(P21, W1), moore_penrose_inverse(S_B))
    Sigma_A = np.diag(Sigma_A)
    alpha = np.sqrt(np.diag(np.dot(Sigma_A.T, Sigma_A)))
    beta = np.sqrt(np.diag(np.dot(S_B.T, S_B)))
    return U, V, alpha, beta


In [218]:
A = np.array([[1, 2, 3, 1, 5],
              [0, 3, 2, 0, 2],
              [1, 0, 2, 1, 0],
              [0, 2, 3, 0, -1],
              [1, 0, 2, 1, 1],
              [0, 2, 1, 0, 1]])


B = np.array([[1, -2, 2, 1, 1],
              [0, 3, 0, 0, 0],
              [1, -2, 2, 1, 1],
              [0, 2, 0, 0, 0],
              [2, -4, 4, 2, 2],
              [1, 3, 2, 1, 1]])

In [219]:
A.shape

(6, 5)

In [220]:
B.shape

(6, 5)

In [221]:
U, V, alpha, beta = GSVD(A, B)

In [ ]:
alpha

In [222]:
alpha

array([1.        , 1.        , 0.57884631, 0.15378845])

In [223]:
beta

array([4.77173113e-17, 8.78515759e-17, 8.15436659e-01, 9.88103797e-01])

In [180]:
np.sqrt(np.diag(Sigma_B))

array([1.12168955e-16, 8.28008025e-01, 9.88149658e-01, 2.41098029e-17,
       7.33392243e-17])

In [195]:
np.sqrt(np.diag(Sigma_A))

array([1.        , 1.        , 1.        , 0.56071625, 0.1534935 ])

In [196]:
np.sqrt(np.diag(Sigma_B))

array([2.41098029e-17, 7.33392243e-17, 1.12168955e-16, 8.28008025e-01,
       9.88149658e-01])

In [198]:
U.shape

(6, 6)

In [199]:
V.shape

(6, 6)